In [1]:
import pandas as pd

def get_sp500_tickers() -> pd.DataFrame:
    """
    Obtiene la lista actualizada de tickers del índice S&P 500
    desde la página oficial de Wikipedia, con encabezado User-Agent.

    Returns
    -------
    pd.DataFrame
        DataFrame con columnas ['symbol', 'name', 'sector'].
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    tables = pd.read_html(url, storage_options=headers)

    df = tables[0]
    df = df.rename(
        columns={
            "Symbol": "symbol",
            "Security": "name",
            "GICS Sector": "sector",
        }
    )

    return df[["symbol", "name", "sector"]]



In [2]:

df_tickers = get_sp500_tickers()
print(df_tickers.head())
print(f"Total tickers: {len(df_tickers)}")


  symbol                 name                  sector
0    MMM                   3M             Industrials
1    AOS          A. O. Smith             Industrials
2    ABT  Abbott Laboratories             Health Care
3   ABBV               AbbVie             Health Care
4    ACN            Accenture  Information Technology
Total tickers: 503


In [3]:
df_tickers

,symbol,name,sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,XYL,Xylem Inc.,Industrials
499,YUM,Yum! Brands,Consumer Discretionary
500,ZBRA,Zebra Technologies,Information Technology
501,ZBH,Zimmer Biomet,Health Care


In [4]:
df_tickers.to_csv("data/sp500_tickers.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data'

In [9]:
import yfinance as yf
import pandas as pd
import time
from tqdm import tqdm

def clean_symbol(symbol):
    return symbol.replace('.', '-')

def fetch_ohlcv_yf(symbol: str, start: str, end: str, max_retries=3) -> pd.DataFrame:
    symbol = symbol.replace('.', '-')
    for attempt in range(1, max_retries + 1):
        try:
            data = yf.download(symbol, start=start, end=end, progress=False, threads=False)
            if data.empty:
                raise ValueError("Empty response")
            data.reset_index(inplace=True)
            data["symbol"] = symbol
            data.rename(
                columns={
                    "Date": "timestamp",
                    "Open": "open",
                    "High": "high",
                    "Low": "low",
                    "Close": "close",
                    "Adj Close": "adj_close",
                    "Volume": "volume",
                },
                inplace=True,
            )
            return data[
                ["timestamp", "symbol", "open", "high", "low", "close", "adj_close", "volume"]
            ]
        except Exception as e:
            print(f"⚠️ [{symbol}] intento {attempt}/{max_retries} fallido: {e}")
            time.sleep(2 ** attempt)  # backoff exponencial
    print(f"🚫 {symbol} falló tras {max_retries} intentos.")
    return pd.DataFrame()



In [10]:
def batch_fetch_data(symbols, start, end, pause=1.0):
    all_data = []
    for symbol in tqdm(symbols, desc="Fetching data"):
        df_symbol = fetch_ohlcv_yf(symbol, start, end)
        if not df_symbol.empty:
            all_data.append(df_symbol)
        time.sleep(pause)
    if not all_data:
        print("⚠️ No se descargaron datos válidos.")
        return pd.DataFrame(columns=["timestamp", "symbol", "open", "high", "low", "close", "adj_close", "volume"])
    return pd.concat(all_data, ignore_index=True)


In [11]:
test_symbols = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA"]
df_prices = batch_fetch_data(test_symbols, "2024-01-01", "2024-03-01", pause=1)
print(df_prices.head())
print(f"Rows: {len(df_prices)} | Symbols: {df_prices['symbol'].nunique()}")



Fetching data:   0%|          | 0/5 [00:00<?, ?it/s]Failed to get ticker 'AAPL' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')


⚠️ [AAPL] intento 1/3 fallido: Empty response


Failed to get ticker 'AAPL' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')


⚠️ [AAPL] intento 2/3 fallido: Empty response


Failed to get ticker 'AAPL' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')


...

⚠️ [GOOG] intento 2/3 fallido: Empty response


Failed to get ticker 'GOOG' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['GOOG']: Exception('%ticker%: No timezone found, symbol may be delisted')


⚠️ [GOOG] intento 3/3 fallido: Empty response
🚫 GOOG falló tras 3 intentos.


Fetching data:  60%|██████    | 3/5 [00:55<00:36, 18.41s/it]


KeyboardInterrupt: 

In [8]:
db_path = "data/market.duckdb"
con = duckdb.connect(db_path)

con.execute("""
CREATE TABLE IF NOT EXISTS ohlcv (
    timestamp TIMESTAMP,
    symbol VARCHAR,
    open DOUBLE,
    high DOUBLE,
    low DOUBLE,
    close DOUBLE,
    adj_close DOUBLE,
    volume DOUBLE
)
""")

# Insertar o reemplazar datos nuevos
con.execute("DELETE FROM ohlcv WHERE symbol IN (SELECT DISTINCT symbol FROM df_prices)")
con.execute("INSERT INTO ohlcv SELECT * FROM df_prices")
con.commit()
con.close()

NameError: name 'duckdb' is not defined

In [ ]:
con = duckdb.connect("data/market.duckdb")
df = con.execute("SELECT symbol, COUNT(*) AS nrows FROM ohlcv GROUP BY symbol").df()
print(df.head())
con.close()